In [1]:
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import pandas as pd
import pickle
import docx
import tkinter as tk
from tkinter import filedialog

In [3]:
# Загрузка модели из файла .h5
model = keras.models.load_model(r'C://Users//user//Desktop//vtb_test//best_model.h5')

In [4]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [5]:
max_review_len = 40

In [17]:
def class_text_tf(text_docx):
    sequence = tokenizer.texts_to_sequences([text_docx])
    data = pad_sequences(sequence, maxlen=max_review_len)
    result = model.predict(data)
    return result

In [ ]:
root = tk.Tk()
root.withdraw()

file_paths = filedialog.askopenfilenames(filetypes=[("Word Documents", "*.docx")])

if file_paths:
    for file_path in file_paths:
        doc = docx.Document(file_path)
        
        s1 = pd.DataFrame(columns=['text']) 
        for i in range(len(doc.paragraphs)):
            s1 = pd.concat([s1, pd.DataFrame([doc.paragraphs[i].text])], ignore_index=True)
        df_ph = pd.DataFrame(s1['text'])
        assigments_table = pd.DataFrame(columns=['assigments_text', 'block_number'])
        list_assigments = df_ph.query('text == "Решения:" or text == "Результаты:"').index.tolist()
        number_of_ranges = len(list_assigments)/2
        number_of_ranges = int(number_of_ranges)

        right_border = [i for i in range(0,len(list_assigments)) if i % 2 != 0]
        left_border = [i for i in range(0,len(list_assigments)) if i % 2 == 0]
        for j in range (number_of_ranges):
            for i in range(list_assigments[left_border [j]]+1 , list_assigments [right_border [j]]-1):
                if len(df_ph.text[i]) > 1 and df_ph.text[i] != '':
                    assigments_table = assigments_table.append({'assigments_text': df_ph.text[i], 'block_number': range(number_of_ranges)[j]+1}, ignore_index=True)

        solutions = assigments_table
        solutions = solutions.assigments_text.to_frame()
        solutions['text_type'] = ''

        coef = []
        text_type = []
        for i in range (len(solutions.assigments_text)):
            coef.append(class_text_tf(solutions.assigments_text[i]))
        solutions['coef'] = coef

        for i in range (len (reshenia.coef)):
            if solutions.coef[i] < 0.5:
                solutions.text_type[i] ="документ_1"
            else:
                solutions.text_type[i] ="документ_2"

        result_table = solutions
        result_table.to_excel('result_table.xlsx')

        result_filename = file_path.replace(".docx", "_result_table.xlsx")
        result_table.to_excel(result_filename)

        print(f"Создан файл: {result_filename}")
        
        
else:
    print("Файлы не выбраны")